<a href="https://colab.research.google.com/github/teploe-odealko/meteoinfo_parsing/blob/main/%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Настройка

In [1]:
hours = 24

In [2]:
hours_button_map = {12: 'id_l_1', 24: 'id_l_2', 36: 'id_l_3'}
try:
    hours_button_id = hours_button_map[hours]
except:
    raise KeyError('Неправильное значение переменной "hours"')

Шаблон meteoinfo (tr_template.xlsx)

In [3]:
from google.colab import files
meteoinfo_filename = files.upload()
meteoinfo_filename = next(iter(meteoinfo_filename))

Saving rt_template_23-07-2021_12.xlsx to rt_template_23-07-2021_12.xlsx


#Скрипт

##install & import

In [4]:
!pip install selenium
!pip install python-pptx
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install pip install excel-formulas-calculator
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 904 kB 5.0 MB/s 
     |████████████████████████████████| 9.3 MB 5.1 MB/s 
     |████████████████████████████████| 149 kB 52.8 MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.19-py3-none-any.whl size=469951 sha256=1b8063eea7cb80019f0fc1677e7662a00729e55429443c0dba793f031687eafa
  Stored in directory: /root/.cache/pip/wheels/17/2f/48/d23f362a5c4728f063b61c4be61e23c2415da1a371f902e5b4
Successfully built python-pptx
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.down

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [5]:
import openpyxl
from typing import Dict, List, Tuple

from bs4 import BeautifulSoup
import urllib3
from openpyxl.worksheet.worksheet import Worksheet
from nltk.stem.snowball import SnowballStemmer
from datetime import datetime, date, time, timedelta
from openpyxl.formula import Tokenizer
from openpyxl.formula.translate import Translator
from openpyxl.utils.cell import get_column_letter
from copy import copy
from openpyxl.formatting.rule import FormulaRule

import re
from openpyxl.styles import Color, PatternFill, Font, Border
from pptx import Presentation
from efc.interfaces.iopenpyxl import OpenpyxlInterface
import numpy as np
urllib3.disable_warnings()

##Meteoinfo погодные явления

In [6]:
!pip install requests-html

     |████████████████████████████████| 87 kB 3.6 MB/s 
     |████████████████████████████████| 76 kB 3.8 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 138 kB 34.7 MB/s 
     |████████████████████████████████| 127 kB 46.3 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13503 sha256=85ff0868890f09a8774fce2dee8ac441921528875ed28ab21f34bbe2df1f11ac
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=5967adc5e0d06b90796f068cd0cbebbe8b6c3ed142e6f24d15f27aaa9d7ae72f
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled ur

In [13]:
#!/usr/bin/env python3
import collections
from typing import Dict, List
from collections import defaultdict
from openpyxl.worksheet.worksheet import Worksheet
import sys
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib3
import requests
import openpyxl

import re
import warnings
import requests
from openpyxl import load_workbook
from datetime import datetime
import requests

warnings.filterwarnings('ignore')
urllib3.disable_warnings()


URLS = [
    'https://meteoinfo.ru/hdmapsmeteoalarmszfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmcfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmprfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmyugfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmyskfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmural',
    'https://meteoinfo.ru/hdmapsmeteoalarmsyb',
    'https://meteoinfo.ru/hdmapsmeteoalarmdv',
]

HAZARDS_URL = 'https://meteoinfo.ru/hazardsbull'

SIZE = 4

SHEET_NAME = 'Билет по регионам'

REGION_COLUMN = 2

FORECAST_COLUMNS = (6, 7, 8, 9)
FACTOR_COLUMNS = (10, 11, 12, 13)

FORCASTS = (
    'Угроза повреждения ЛЭП и линий связи, обрушения слабо укрепленных, широкоформатных, ветхих и рекламных конструкций',
    'Угроза выхода из строя систем ЖКХ, городских коммуникаций',
    'Угроза затопления населенных пунктов, дорог, мостов',
    'Угроза сбоев в работе всех видов транспорта, увеличение количества ДТП',
    'Угроза жизни и здоровью населения',
    'Увеличение количества травм пешеходов',
    'Угроза повреждения населенных пунктов, дорог, мостов'
)

# INTERCHANGEABLE_FACTORS = [
#     {'Ветер', 'Дождь', 'Снег/Обледенение', 'Гроза'},
#     {'Заморозки', 'Очень низкая температура'},
#     {'Туман', 'Голеледно - изморозевые отложения'},
#     {'Пожарная опасность'},
#     {'Паводок'},
#     {'Очень высокая температура'}
# ]

FORCAST_MAP = {
    'очень высокая температура': set((FORCASTS[4],)),
    'очень низкая температура': set((FORCASTS[1], FORCASTS[4])),
    'заморозки': set((FORCASTS[1], FORCASTS[4])),
    'ветер': set((FORCASTS[0], FORCASTS[1], FORCASTS[4])),
    'дождь': set((FORCASTS[0], FORCASTS[1])),
    'гроза': set((FORCASTS[0], FORCASTS[1])),
    'град': set((FORCASTS[0], FORCASTS[1])),
    'паводок': set((FORCASTS[2], FORCASTS[4])),
    'наводнение': set((FORCASTS[2], FORCASTS[4])),
    'снег/Обледенение': set((FORCASTS[0], FORCASTS[1], FORCASTS[3])),
    'голеледно - изморозевые отложения': set((FORCASTS[3], FORCASTS[5])),
    'лавины': set((FORCASTS[4], FORCASTS[6])),
    'сель': set((FORCASTS[4], FORCASTS[6])),
    'прибрежные события': set((FORCASTS[1], FORCASTS[6], FORCASTS[4])),
    'пыльная (песчаная) буря': set((FORCASTS[0], FORCASTS[1], FORCASTS[3], FORCASTS[4])),
    'туман': set((FORCASTS[3],)),
    'пожарная опасность': set((FORCASTS[6], FORCASTS[4])),
    'прочие опасности': set(),
}

DASHBOARD_LABEL_MAP = {
    'очень высокая температура': 'Критическое повышение температуры воздуха',
    'очень низкая температура': 'Критическое понижение температуры воздуха',
    'заморозки': 'Заморозки',
    'ветер': 'Сильный ветер',
    'дождь': 'Дождь',
    'гроза': 'Гроза',
    'паводок': 'Паводок, подтопление',
    'наводнение': 'Наводнение',
    'снег/Обледенение': 'Снегопад, метель',
    'голеледно - изморозевые отложения': 'Гололедица',
    'лавины': 'Оползни, обвалы, снежные лавины',
    'сель': 'Сель',
    'прибрежные события': 'Прибрежные события',
    'пыльная (песчаная) буря': 'Пыльная (песчаная) буря',
    'туман': 'Туман',
    'пожарная опасность': 'Пожароопасная обстановка, пожары',
    'прочие опасности': 'Прочие опасности',
    'град': 'Град или снежный дождь'
}

FACTORS = FORCAST_MAP.keys()

regions_need_to_join = {'саха',
                        'магаданская',
                        'ямало ненецкий ао',
                        'камчатский край',
                        'амурская',
                        'хабаровский край',
                        'чеч',
                        'северная осетия',
                        'красноярский',
                        'моск'}


def join_region(region: str) -> str:
    if region.startswith('сахалин'):
        return 'сахалин'
    for reg in regions_need_to_join:
        if region.startswith(reg):
            return reg
    return region


def normolize_string(string: str):
    string = string.strip().lower().translate({ord('.'): None})
    string = ' '.join(string.split('-'))

    # remove stop words
    stop_words = ['г', 'обл', 'область', 'республика', 'югра', 'ао']
    try:
        splitted_string = string.split()
    except IndexError:
        return ''
    string = ' '.join(w for w in splitted_string if w not in stop_words)
    return join_region(string)


def remove_interchangeable_factors(factor_desc_list: list):
    factors_set = set()
    prev_len_factors_set = 0

    for factor_desc in factor_desc_list:
        factor = factor_desc[0]
        forecast = FORCAST_MAP.get(factor.lower())
        if not forecast:
            continue
        factors_set.update(forecast)
        if len(factors_set) == prev_len_factors_set:
            factor_desc_list.remove(factor_desc)
            return factor_desc_list
        prev_len_factors_set = len(factors_set)
    return factor_desc_list


def analyze_factors(factors_desc: list):
    final_forecasts = set()

    prev_len = len(factors_desc)
    while len(factors_desc) > SIZE:
        factors_desc = remove_interchangeable_factors(factors_desc)
        if len(factors_desc) == prev_len:  # If nothing has been deleted, means there is no interchangeable factors
            factors_desc = factors_desc[:SIZE]
        prev_len = len(factors_desc)

    for factor, desc in factors_desc:
        forecast: set = FORCAST_MAP.get(factor.lower())
        if forecast is None:
            print(f"\x1b[31mНЕИЗВЕСТНЫЙ ФАКТОР \"{factor}\"\x1b[0m")
            continue
        final_forecasts.update(forecast)

    while len(final_forecasts) > SIZE:
        final_forecasts.pop()

    return final_forecasts, set(factors_desc)


def factor_mapping(factor: str, description: str):
    dashboard_factor = DASHBOARD_LABEL_MAP.get(factor.lower())
    if dashboard_factor == 'Дождь':
        for word in description.split():
            if word.startswith('сильн') or word.startswith('лив'):
                dashboard_factor = 'Сильный дождь, ливневой дождь'
                break
    return dashboard_factor

def fill_excel_regions_weather(ws: Worksheet, regions: dict):
    for row in range(2, ws.max_row + 1):
        name = ws.cell(row=row, column=REGION_COLUMN).value
        if not name:
            continue

        name = normolize_string(name)
        factors_desc = regions.get(name)

        if not factors_desc: # if there is no forecast for the region
            # print(name)
            continue

        forecasts, factors_desc = analyze_factors(factors_desc)
        for column, forecast in zip(FORECAST_COLUMNS, forecasts):
            ws.cell(row=row, column=column).value = forecast

        for column, factor_desc in zip(FACTOR_COLUMNS, factors_desc):
            factor = factor_desc[0]
            desc = factor_desc[1]
            ws.cell(row=row, column=column).value = factor_mapping(factor, desc)


def get_hazards(ws:Worksheet, url: str):
    session = HTMLSession()
    response = session.get(url, verify=False)
    rows = response.html.find('#div_1', first=True).xpath('*/table/tbody/tr')
    row_data = rows[1].xpath('//p', first=True)
    ws['B5'].value = row_data.text
    # print(row_data.text)


def parse_description(factor_list: list, region_factor_desc: str):
    for word in re.split(';|,|\.| ', region_factor_desc):
        # try:
        #     factors = list(zip(*factor_list))[0]
        # except IndexError:
        #     factors = []
        if (word.lower().startswith('дожд') or word.lower().startswith('лив')):
            factor_list.append(('Дождь', region_factor_desc))
        elif word.lower().startswith('гроз'):
            factor_list.append(('Гроза', region_factor_desc))
        elif word.lower().startswith('град'):
            factor_list.append(('Град', region_factor_desc))
        elif (word.lower().startswith('шквал') or word.lower().startswith('вет')):
            factor_list.append(('Ветер', region_factor_desc))


def get_regions(urls: List[str]) -> Dict:
    # factors = defaultdict(list)
    # descriptions = defaultdict(list)
    regions = defaultdict(list)
    for url in urls:
        wd.get(url)
        hours24_button = wd.find_element_by_id(hours_button_id)
        hours24_button.click()
        for row in wd.find_elements_by_xpath('//div[@class="test"]'):
            try:
                row_data_bs = BeautifulSoup(row.get_attribute('data-original-title'),
                                            features='lxml')
            except:
                print('error')
                continue  # if raw_data is None
            region_name = normolize_string(row_data_bs.find(class_='cl_tbl_obl_name_tooltip').text)
            region_factor = row_data_bs.find(class_='cl_tbl_tooltip_text').find('b').text.strip('. ')
            region_factor_desc = '.'.join(row_data_bs.find(class_='cl_tbl_tooltip_text').text.split('.')[1:]).strip()

            if (region_factor == 'КМЯ' or region_factor == 'Прочие опасности'):
                parse_description(regions[region_name], region_factor_desc)
                # print(regions[region_name], region_factor)
                continue

            

            try:
                factors = list(zip(*regions[region_name]))[0]
            except IndexError:
                factors = []

            if region_factor != '':
                regions[region_name].append((region_factor, region_factor_desc))

    return regions


def fill_excel_covid_rt(ws: Worksheet):
    obl_pattern = re.compile(r'\bобл\.', re.IGNORECASE)
    ao_pattern = re.compile(r'(?<!Еврейская)\bао\b', re.IGNORECASE)
    gorod_pattern = re.compile(r'\bг\.', re.IGNORECASE)
    strange_pattern = re.compile(r'[—]')

    url = 'https://yastat.net/s3/milab/2020/covid19-stat/data/v10/deep_data.json?v=1618916098074'
    sheet = 'Билет по регионам'

    json_dict = requests.get(url, verify=False).json()

    region_rt_dict = {strange_pattern.sub('-', x['info']['name'].lower()): x['info'].get('rt') for x in json_dict['russia_stat_struct']['data'].values()}
    region_rt_dict_short_name = {strange_pattern.sub('-', x['info']['short_name'].lower()): x['info'].get('rt') for x in json_dict['russia_stat_struct']['data'].values()}

    for row in range(2, ws.max_row + 1):
        # print(ws.cell(row=row, column=2).value)
        key = ws.cell(row=row, column=2).value.lower()

        key = obl_pattern.sub('область', key).strip()
        key = gorod_pattern.sub('', key).strip()

        value = region_rt_dict_short_name.get(key, region_rt_dict.get(ao_pattern.sub('автономный округ', key).strip()))

        ws.cell(row=row, column=3).value = round(value, 2) if value else value

def clear_table(ws: Worksheet):
    for row in range(2, ws.max_row + 1):
        for column in (FORECAST_COLUMNS + FACTOR_COLUMNS + (3,)):
            ws.cell(row=row, column=column).value = None

In [14]:
def get_regions_info() -> dict:
    final_region_data = {}

    response: dict = requests.get(URL, verify=False).json()
    regions = response['russia_stat_struct']['data']
    dates = response['russia_stat_struct']['dates']
    for region in regions:
        full_name = regions[region]['info']['name']
        short_name = regions[region]['info']['short_name']
        regions[region]['info'].pop('name', None)
        regions[region]['info'].pop('short_name', None)
        final_region_data[short_name] = regions[region]
    return final_region_data

In [19]:
URL = 'https://yastat.net/s3/milab/2020/covid19-stat/data/v10/deep_data.json'

def situation_in_the_bank(wb):
    regions = get_regions_info()

    # interface = OpenpyxlInterface(wb=wb, use_cache=True)
    day_infections = regions['Россия']['info']['cases_delta']
    rt = wb['Билет по регионам']['C'][2:85]
    rt_bigger_1_num = len([r.value for r in rt if r.value > 1])
    s1 = "субъекте" if (rt_bigger_1_num%10 == 1) and rt_bigger_1_num != 11 else "субъектах"

    gain = []

    regions_dict = {}
    for region in regions:
        if region not in ['Санкт-Петербург', 'Московская область', 'Москва', 'Россия']:
            regions_dict[region] = regions[region]['info']['cases_delta']
    top5 = sorted(regions_dict.items(), key=lambda x: 1/x[1])[:5]

    s2 = ', '.join([f'{region} ({gain})' for region, gain in top5])
    template = f"{day_infections} новых случаев в России. Ухудшение – в {rt_bigger_1_num} {s1} РФ. Наибольший прирост новых случаев: {s2}."
    # print(template)
    wb['Обстановка в банке']['B7'] = template


In [20]:
def meteoinfo(filename: str):
    wb = openpyxl.load_workbook(filename)
    ws = wb[SHEET_NAME]

    clear_table(ws)
    fill_excel_covid_rt(ws=ws)
    regions = get_regions(urls=URLS)
    # print(regions)
    # remove duplicate factors
    for region in regions:
        factors = regions[region]
        final_list = []
        for factor in factors:
            try:
                unique_factors = list(zip(*final_list))[0]
            except:
                unique_factors = []
            if factor[0] not in unique_factors:
                final_list.append(factor)
        regions[region] = final_list


    fill_excel_regions_weather(ws=ws, regions=regions)

    # B5
    get_hazards(wb['Обстановка в банке'], HAZARDS_URL)
    # B7
    situation_in_the_bank(wb)

    today = datetime.today().strftime("%d-%m-%Y")
    splitted_filename = filename.split('.')
    splitted_filename[0] = f'{splitted_filename[0]}_{today}_{hours}'
    output_file_name = '.'.join(splitted_filename)

    wb.save(output_file_name)
    return output_file_name

##main

In [25]:
def main(meteoinfo_filename: str):
    # Заполнение погодных данных с meteoinfo в xlsx файл
    return meteoinfo(meteoinfo_filename)
    



# Результат

In [26]:
out_file = main(meteoinfo_filename)

In [27]:
files.download(out_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>